In [3]:
import pandas as pd

# Read x_train.txt
X = pd.read_csv('../data/x_train.txt', sep=' ', header=None)

# Read y_train.txt
y = pd.read_csv('../data/y_train.txt', sep=' ', header=None)


# Display the data
print("x:")
print(X.head())

print("\ny:")
print(y.head())


x:
        0         1         2         3         4         5         6    \
0 -2.619773 -2.619533 -1.199350 -1.083335 -1.000910 -0.366967 -2.164037   
1 -1.415579 -1.782544 -2.880270 -1.958863  1.159968  0.273030 -1.628728   
2 -2.745092 -1.382945 -1.626015 -1.282560 -0.663146  0.052349 -2.403322   
3  0.618998  0.455364 -0.115081  0.649040 -0.862207  2.308504  0.526114   
4 -0.070694 -0.550509 -0.565556 -0.693065 -0.573089 -0.395862  0.003170   

        7         8         9    ...        490        491        492  \
0 -1.210001 -0.658311 -1.489539  ...  10.849925  10.343346  10.717519   
1 -0.175813 -0.916857 -0.570166  ...  11.489417   5.195818   3.494627   
2 -0.765073 -0.394354 -0.806624  ...  13.934934   9.267515   4.705604   
3 -1.094852  1.088656 -0.481210  ...  12.021328   3.852231  11.059702   
4 -0.981609 -0.505775 -0.758430  ...   7.537788  11.229665  11.318915   

        493        494        495        496        497        498        499  
0  7.709295   5.894554  12.

In [4]:
import ray
from ray import tune
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.search.bohb import TuneBOHB
from ray.train import report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH
import ast
# Assuming your data is in X and y dataframes

def train_ensemble(config):
    vars = ast.literal_eval(config['colset'])
    Xloc = X[vars]
    # Split data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(Xloc, y, test_size=0.2)
   
    
    # Initialize the models with the given hyperparameters
    xgb_model_1 = XGBClassifier(
        eta=config["xgb_eta"],
        max_depth=config["xgb_max_depth"],
        min_child_weight=config["xgb_min_child_weight"],
        subsample=config["xgb_subsample"],
        colsample_bytree=config["xgb_colsample_bytree"],
        reg_lambda=config["xgb_lambda"],
        reg_alpha=config["xgb_alpha"],
        objective="binary:logistic",
        eval_metric="logloss",
        use_label_encoder=False,
        verbosity=0
    )

    xgb_model_2 = XGBClassifier(
        eta=config["xgb_eta_2"],
        max_depth=config["xgb_max_depth_2"],
        min_child_weight=config["xgb_min_child_weight_2"],
        subsample=config["xgb_subsample_2"],
        colsample_bytree=config["xgb_colsample_bytree_2"],
        reg_lambda=config["xgb_lambda_2"],
        reg_alpha=config["xgb_alpha_2"],
        objective="binary:logistic",
        eval_metric="logloss",
        use_label_encoder=False,
        verbosity=0
    )
    xgb_model_3 = XGBClassifier(
        eta=config["xgb_eta_3"],
        max_depth=config["xgb_max_depth_3"],
        min_child_weight=config["xgb_min_child_weight_3"],
        subsample=config["xgb_subsample_3"],
        colsample_bytree=config["xgb_colsample_bytree_3"],
        reg_lambda=config["xgb_lambda_3"],
        reg_alpha=config["xgb_alpha_3"],
        objective="binary:logistic",
        eval_metric="logloss",
        use_label_encoder=False,
        verbosity=0
    )

    et_model_1 = ExtraTreesClassifier(
        n_estimators=config["et_n_estimators"],
        max_depth=config["et_max_depth"],
        min_samples_split=config["et_min_samples_split"],
        min_samples_leaf=config["et_min_samples_leaf"],
        max_features=config["et_max_features"],
        bootstrap=config["et_bootstrap"]
    )

    et_model_2 = ExtraTreesClassifier(
        n_estimators=config["et_n_estimators_2"],
        max_depth=config["et_max_depth_2"],
        min_samples_split=config["et_min_samples_split_2"],
        min_samples_leaf=config["et_min_samples_leaf_2"],
        max_features=config["et_max_features_2"],
        bootstrap=config["et_bootstrap_2"]
    )

    et_model_3 = ExtraTreesClassifier(
        n_estimators=config["et_n_estimators_3"],
        max_depth=config["et_max_depth_3"],
        min_samples_split=config["et_min_samples_split_3"],
        min_samples_leaf=config["et_min_samples_leaf_3"],
        max_features=config["et_max_features_3"],
        bootstrap=config["et_bootstrap_3"]
    )
    
    svc_model_1 = SVC(
        C=config["svc_C"],
        kernel=config["svc_kernel"],
        probability=True
    )

    svc_model_2 = SVC(
        C=config["svc_C_2"],
        kernel=config["svc_kernel_2"],
        probability=True
    )

    svc_model_3 = SVC(
        C=config["svc_C_3"],
        kernel=config["svc_kernel_3"],
        probability=True
    )

    lr_model_1 = LogisticRegression(
        C=config["lr_C"],
        solver=config["lr_solver"],
        max_iter=1000
    )

    lr_model_2 = LogisticRegression(
        C=config["lr_C_2"],
        solver=config["lr_solver_2"],
        max_iter=1000
    )

    lr_model_3 = LogisticRegression(
        C=config["lr_C_3"],
        solver=config["lr_solver_3"],
        max_iter=1000
    )

    ensemble_model = VotingClassifier(
        estimators=[
            ('xgb_1', xgb_model_1),
            ('xgb_2', xgb_model_2),
            ('xgb_3', xgb_model_3),
            ('et_1', et_model_1),
            ('et_2', et_model_2),
            ('et_3', et_model_3),
            ('svc_1', svc_model_1),
            ('svc_2', svc_model_2),
            ('svc_3', svc_model_3),
            ('lr_1', lr_model_1),
            ('lr_2', lr_model_2),
            ('lr_3', lr_model_3)
        ],
        voting='soft',
        weights=[
            config["weight_xgb"],
            config["weight_xgb"],
            config["weight_xgb"],
            config["weight_et"],
            config["weight_et"],
            config["weight_et"],
            config["weight_svc"],
            config["weight_svc"],
            config["weight_svc"],
            config["weight_lr"],
            config["weight_lr"],
            config["weight_lr"]
        ]
    )









    ensemble_model.fit(X_train, y_train)
    
    # Ensemble the predictions by averaging
    #ensemble_preds_proba = (xgb_preds_proba + et_preds_proba + svc_preds_proba + lr_preds_proba) / 4
    ensemble_preds_proba = ensemble_model.predict_proba(X_val)[:, 1]
    
    # Get the indices of the top 20% predictions
    top_20_percent_indices = np.argsort(ensemble_preds_proba)[-int(0.2 * len(ensemble_preds_proba)):]

    # Select the corresponding true labels and predicted labels for top 20%
    top_20_true = y_val.iloc[top_20_percent_indices].values
    top_20_preds = (ensemble_preds_proba[top_20_percent_indices] >= 0.5).astype(int)
    
    # Calculate the number of correct class 1 predictions
    correct_class_1_predictions = sum(1 for true, pred in zip(top_20_true, top_20_preds) if true == 1 and pred == 1)
    
    # Calculate the number of features used
    num_features_used = Xloc.shape[1]

    #scaled score calculation
    customer_scaled = (correct_class_1_predictions / len(top_20_preds)) * 1000
    customer_gain = 10 * customer_scaled
    variable_cost = 200 * num_features_used
    custom_score = customer_gain - variable_cost
    
    # Compute the custom score
    #custom_score = (correct_class_1_predictions * 10) - (num_features_used * 200)
    
    report({"custom_score": custom_score})

# Define the search space using ConfigSpace
config_space = CS.ConfigurationSpace()

# Hyperparameters for XGBoost
config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('xgb_eta', lower=0.01, upper=0.1, log=True))
config_space.add_hyperparameter(CSH.UniformIntegerHyperparameter('xgb_max_depth', lower=3, upper=10))
config_space.add_hyperparameter(CSH.UniformIntegerHyperparameter('xgb_min_child_weight', lower=1, upper=5))
config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('xgb_subsample', lower=0.5, upper=1.0))
config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('xgb_colsample_bytree', lower=0.5, upper=1.0))
config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('xgb_lambda', lower=1e-3, upper=10.0, log=True))
config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('xgb_alpha', lower=1e-3, upper=10.0, log=True))

config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('xgb_eta_2', lower=0.05, upper=0.9, log=True))
config_space.add_hyperparameter(CSH.UniformIntegerHyperparameter('xgb_max_depth_2', lower=2, upper=5))
config_space.add_hyperparameter(CSH.UniformIntegerHyperparameter('xgb_min_child_weight_2', lower=1, upper=10))
config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('xgb_subsample_2', lower=0.2, upper=1.0))
config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('xgb_colsample_bytree_2', lower=0.5, upper=1.0))
config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('xgb_lambda_2', lower=1e-4, upper=5, log=True))
config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('xgb_alpha_2', lower=1e-4, upper=5, log=True))


config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('xgb_eta_3', lower=0.5, upper=1, log=True))
config_space.add_hyperparameter(CSH.UniformIntegerHyperparameter('xgb_max_depth_3', lower=1, upper=20))
config_space.add_hyperparameter(CSH.UniformIntegerHyperparameter('xgb_min_child_weight_3', lower=1, upper=5))
config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('xgb_subsample_3', lower=0.001, upper=1.0))
config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('xgb_colsample_bytree_3', lower=0.2, upper=1.0))
config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('xgb_lambda_3', lower=1e-3, upper=10.0, log=True))
config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('xgb_alpha_3', lower=1e-3, upper=10.0, log=True))

# Hyperparameters for ExstraTrees
config_space.add_hyperparameter(CSH.UniformIntegerHyperparameter('et_n_estimators', lower=10, upper=100))
config_space.add_hyperparameter(CSH.UniformIntegerHyperparameter('et_max_depth', lower=1, upper=50))
config_space.add_hyperparameter(CSH.UniformIntegerHyperparameter('et_min_samples_split', lower=2, upper=20))
config_space.add_hyperparameter(CSH.UniformIntegerHyperparameter('et_min_samples_leaf', lower=1, upper=20))
config_space.add_hyperparameter(CSH.CategoricalHyperparameter('et_max_features', ['sqrt', 'log2']))
config_space.add_hyperparameter(CSH.CategoricalHyperparameter('et_bootstrap', [True, False]))

config_space.add_hyperparameter(CSH.UniformIntegerHyperparameter('et_n_estimators_2', lower=10, upper=200))
config_space.add_hyperparameter(CSH.UniformIntegerHyperparameter('et_max_depth_2', lower=2, upper=30))
config_space.add_hyperparameter(CSH.UniformIntegerHyperparameter('et_min_samples_split_2', lower=5, upper=30))
config_space.add_hyperparameter(CSH.UniformIntegerHyperparameter('et_min_samples_leaf_2', lower=2, upper=30))
config_space.add_hyperparameter(CSH.CategoricalHyperparameter('et_max_features_2', ['sqrt', 'log2']))
config_space.add_hyperparameter(CSH.CategoricalHyperparameter('et_bootstrap_2', [True, False]))

config_space.add_hyperparameter(CSH.UniformIntegerHyperparameter('et_n_estimators_3', lower=50, upper=200))
config_space.add_hyperparameter(CSH.UniformIntegerHyperparameter('et_max_depth_3', lower=1, upper=40))
config_space.add_hyperparameter(CSH.UniformIntegerHyperparameter('et_min_samples_split_3', lower=10, upper=40))
config_space.add_hyperparameter(CSH.UniformIntegerHyperparameter('et_min_samples_leaf_3', lower=3, upper=30))
config_space.add_hyperparameter(CSH.CategoricalHyperparameter('et_max_features_3', ['sqrt', 'log2']))
config_space.add_hyperparameter(CSH.CategoricalHyperparameter('et_bootstrap_3', [True, False]))

# Hyperparameters for SVC
config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('svc_C', lower=0.01, upper=10.0, log=True))
config_space.add_hyperparameter(CSH.CategoricalHyperparameter('svc_kernel', ['linear', 'rbf', 'poly', 'sigmoid']))

config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('svc_C_2', lower=0.1, upper=20.0, log=True))
config_space.add_hyperparameter(CSH.CategoricalHyperparameter('svc_kernel_2', ['linear', 'rbf', 'poly', 'sigmoid']))

config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('svc_C_3', lower=0.1, upper=30.0, log=True))
config_space.add_hyperparameter(CSH.CategoricalHyperparameter('svc_kernel_3', ['linear', 'rbf', 'poly', 'sigmoid']))

# Hyperparameters for Logistic Regression
config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('lr_C', lower=0.1, upper=10.0, log=True))
config_space.add_hyperparameter(CSH.CategoricalHyperparameter('lr_solver', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']))

config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('lr_C_2', lower=0.1, upper=20.0, log=True))
config_space.add_hyperparameter(CSH.CategoricalHyperparameter('lr_solver_2', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']))

config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('lr_C_3', lower=0.1, upper=30.0, log=True))
config_space.add_hyperparameter(CSH.CategoricalHyperparameter('lr_solver_3', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']))

# Weights

config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('weight_xgb', lower=0.0, upper=1.0))
config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('weight_et', lower=0.0, upper=1.0))
config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('weight_svc', lower=0.0, upper=1.0))
config_space.add_hyperparameter(CSH.UniformFloatHyperparameter('weight_lr', lower=0.0, upper=1.0))


config_space.add_hyperparameter(CSH.CategoricalHyperparameter('colset', ['[101, 102, 103, 105]', '[101, 102, 103]', '[101, 102, 105]', '[101, 103, 105]', '[102, 103, 105]', '[104, 102, 103, 105]', '[100, 101, 102, 103, 105]', '[100, 101, 102, 103]', '[100, 101, 102, 105]', '[100, 101, 103, 105]', '[100, 102, 103, 105]', '[100, 104, 102, 103, 105]', '[100, 101, 102, 103, 104, 105]']))

# Set up the BOHB search algorithm
bohb_search = TuneBOHB(config_space, metric="custom_score", mode="max")

# Set up the HyperBandForBOHB scheduler
bohb_scheduler = HyperBandForBOHB(
    time_attr="training_iteration",
    metric="custom_score",
    mode="max"
)

# Function to create shorter directory names
def trial_dirname_creator(trial):
    return f"trial_{trial.trial_id}"

# Run the hyperparameter search
analysis = tune.run(
    train_ensemble,
    resources_per_trial={"cpu": 1, "gpu": 0},
    search_alg=bohb_search,
    scheduler=bohb_scheduler,
    num_samples=500,
    trial_dirname_creator=trial_dirname_creator,
    raise_on_failed_trial=False
)

# Get the best result
best_config = analysis.get_best_config(metric="custom_score", mode="max")
print("Best config: ", best_config)


2024-06-02 20:50:31,882	INFO tune.py:614 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


Trial name,custom_score
train_ensemble_0048c679,6250
train_ensemble_00c927dd,6500
train_ensemble_0135d205,6600
train_ensemble_01448e40,6600
train_ensemble_0186ecad,6850
train_ensemble_019f41aa,6650
train_ensemble_02012eaa,7000
train_ensemble_02b6bbe2,7300
train_ensemble_035882f4,6900
train_ensemble_037be88a,5700


2024-06-02 20:53:17,244	INFO hyperband.py:543 -- Restoring from a previous point in time. Previous=1; Now=1
2024-06-02 20:53:20,189	INFO hyperband.py:543 -- Restoring from a previous point in time. Previous=1; Now=1
2024-06-02 20:53:21,719	INFO hyperband.py:543 -- Restoring from a previous point in time. Previous=1; Now=1
2024-06-02 20:53:23,503	INFO hyperband.py:543 -- Restoring from a previous point in time. Previous=1; Now=1
2024-06-02 20:53:25,685	INFO hyperband.py:543 -- Restoring from a previous point in time. Previous=1; Now=1
2024-06-02 20:53:26,888	INFO hyperband.py:543 -- Restoring from a previous point in time. Previous=1; Now=1
2024-06-02 20:53:27,787	INFO hyperband.py:543 -- Restoring from a previous point in time. Previous=1; Now=1
2024-06-02 20:53:29,942	INFO hyperband.py:543 -- Restoring from a previous point in time. Previous=1; Now=1
2024-06-02 20:53:32,654	INFO hyperband.py:543 -- Restoring from a previous point in time. Previous=1; Now=1
2024-06-02 20:53:34,116	INFO

Best config:  {'colset': '[101, 102, 103]', 'et_bootstrap': False, 'et_bootstrap_2': True, 'et_bootstrap_3': True, 'et_max_depth': 3, 'et_max_depth_2': 22, 'et_max_depth_3': 13, 'et_max_features': 'log2', 'et_max_features_2': 'log2', 'et_max_features_3': 'sqrt', 'et_min_samples_leaf': 16, 'et_min_samples_leaf_2': 4, 'et_min_samples_leaf_3': 16, 'et_min_samples_split': 10, 'et_min_samples_split_2': 5, 'et_min_samples_split_3': 37, 'et_n_estimators': 73, 'et_n_estimators_2': 85, 'et_n_estimators_3': 105, 'lr_C': 1.5762775359668812, 'lr_C_2': 0.5653236130578693, 'lr_C_3': 0.6256017216185775, 'lr_solver': 'saga', 'lr_solver_2': 'liblinear', 'lr_solver_3': 'sag', 'svc_C': 0.5339032162398551, 'svc_C_2': 0.783003844750391, 'svc_C_3': 0.5872141119130093, 'svc_kernel': 'linear', 'svc_kernel_2': 'linear', 'svc_kernel_3': 'rbf', 'weight_et': 0.8638705586914847, 'weight_lr': 0.6683984883538573, 'weight_svc': 0.9970313409953824, 'weight_xgb': 0.20564442974341401, 'xgb_alpha': 0.02796134913866195, '

In [5]:
df=analysis.dataframe()
df.to_csv('ensemble3-raytune-bohb.csv')

In [6]:
df.sort_values(by='custom_score', ascending=False, inplace=True)
print(df.head())

     custom_score   timestamp checkpoint_dir_name   done  training_iteration  \
128        7750.0  1717354508                None  False                   1   
451        7500.0  1717355134                None  False                   1   
477        7500.0  1717355184                None  False                   1   
322        7400.0  1717354893                None  False                   1   
113        7400.0  1717354480                None  False                   1   

     trial_id                 date  time_this_iter_s  time_total_s    pid  \
128  1ccab6db  2024-06-02_20-55-08          2.982811      2.982811  65716   
451  7ad7d84e  2024-06-02_21-05-34          4.209845      4.209845  41176   
477  8cb3507f  2024-06-02_21-06-24          3.808508      3.808508  66464   
322  37c5878e  2024-06-02_21-01-33          4.066246      4.066246  47172   
113  b82e0c21  2024-06-02_20-54-40          4.920715      4.920715  44380   

     ... config/xgb_max_depth config/xgb_max_depth_2  co